In [1]:
import datetime
import random

import numpy as np
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm

from common import *

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print(".. setting base configuration & params ..")

initialize()

best_model_filepath = './saved-models/achitecture-01.hdf5'
training_epochs = 50
batch_size = 20
images_size = 270


.. setting base configuration & params ..


In [3]:
(train_tensors, X_train, y_train,
test_tensors, X_test, y_test, 
valid_tensors, X_validate, y_validate, 
duration_loading) = load_and_split_data(images_size=images_size)

  0%|          | 5/1408 [00:00<00:28, 49.13it/s]

.. loading & splitting data ..


100%|██████████| 352/352 [00:03<00:00, 95.99it/s] 


In [4]:
print(".. constructing the model ..")

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', input_shape=(images_size, images_size, 3), 
                 data_format="channels_last"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


.. constructing the model ..
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 270, 270, 16)      208       
_________________________________________________________________
batch_normalization_1 (Batch (None, 270, 270, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 270, 270, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 135, 135, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 135, 135, 32)      2080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 135, 135, 32)      128       
_________________________________________________________________
activation_2 (Activation)    (None, 135, 135, 3

In [5]:
duration_training = train_single_model(model, 
                                       best_model_filepath, 
                                       train_tensors, y_train, 
                                       valid_tensors, y_validate, 
                                       training_epochs, 
                                       batch_size)


.. training the model ..
Train on 1408 samples, validate on 352 samples
Epoch 1/1
1408/1408 [==============================] - 268s 190ms/step - loss: 0.4366 - acc: 0.7820 - val_loss: 0.3859 - val_acc: 0.8097

Epoch 00001: val_loss improved from inf to 0.38588, saving model to ./saved-models/achitecture-01.hdf5


In [6]:
print(".. loading best weights ..")

model.load_weights(best_model_filepath)

.. loading best weights ..


In [7]:
print(".. testing the model on Test data ..")
section_start_time = datetime.datetime.utcnow()

test_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(test_tensors)]

test_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/440 [00:00<?, ?it/s]

.. testing the model on Test data ..


100%|██████████| 440/440 [00:34<00:00, 12.68it/s]


In [8]:
print(".. testing the model on Training data ..")
section_start_time = datetime.datetime.utcnow()

train_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(train_tensors)]

train_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 1/1408 [00:00<02:47,  8.42it/s]

.. testing the model on Training data ..


100%|██████████| 1408/1408 [01:45<00:00, 13.32it/s]


In [9]:
print(".. testing the model on Validation data ..")
section_start_time = datetime.datetime.utcnow()

validation_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(valid_tensors)]

validation_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 1/352 [00:00<00:41,  8.56it/s]

.. testing the model on Validation data ..


100%|██████████| 352/352 [00:26<00:00, 13.49it/s]


In [10]:
print("________________________________________________________")
print("TIMING SUMMARY:\n")
print("loading duration: {0:.1f} seconds".format(duration_loading))
print("training duration: {0:.1f} seconds".format(duration_training))

print("________________________________________________________")
print("MODEL PERFORMANCE ON TEST DATA:\n")
print("predict duration: {0:.1f} seconds".format(test_data_predictions_time))
false_positive_images, false_negative_images = summarize_model_performance(X_test, y_test, test_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON TRAIN DATA:\n")
print("predict duration: {0:.1f} seconds".format(train_data_predictions_time))
_, _ = summarize_model_performance(X_train, y_train, train_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON VALIDATION DATA:\n")
print("predict duration: {0:.1f} seconds".format(validation_data_predictions_time))
_, _ = summarize_model_performance(X_validate, y_validate, validation_data_predictions)

________________________________________________________
TIMING SUMMARY:

loading duration: 25.2 seconds
training duration: 269.0 seconds
________________________________________________________
MODEL PERFORMANCE ON TEST DATA:

predict duration: 34.7 seconds
all:  440
all_positives:  81
all_negatives:  359
true_positives:  8
true_negatives:  356
false_positives:  3
false_negatives:  73
RECALL: 0.10
SPECIFICITY: 0.99
ACCURACY: 0.83
PRECISION: 0.73
F1 SCORE: 0.17
FP RATE / ERROR I: 0.01
FN RATE / ERROR II: 0.90
________________________________________________________
MODEL PERFORMANCE ON TRAIN DATA:

predict duration: 105.7 seconds
all:  1408
all_positives:  316
all_negatives:  1092
true_positives:  21
true_negatives:  1082
false_positives:  10
false_negatives:  295
RECALL: 0.07
SPECIFICITY: 0.99
ACCURACY: 0.78
PRECISION: 0.68
F1 SCORE: 0.12
FP RATE / ERROR I: 0.01
FN RATE / ERROR II: 0.93
________________________________________________________
MODEL PERFORMANCE ON VALIDATION DATA:

pre

## False Positives (first 20):

In [11]:
display_images(false_positive_images[:20])

## False Negatives (first 20):

In [12]:
display_images(false_negative_images[:20])